### Inverse Problem: 1D Euler

An **inverse problem** asks you to “turn the tables” on the usual forward simulation:

> **“Given sparse or noisy observations of a system *and* the governing equations, infer unknown fields or parameters.”**

We'll consider, for this second part, the 1D Euler setting. As such, in a 1D Euler inverse problem, that means:

1. **Known physics**:
  You still trust the Euler equations

  $$
    \begin{aligned}
      &\partial_t \rho + \partial_x(\rho\,u) = 0,\\
      &\partial_t (\rho u) + \partial_x(\rho u^2 + p) = 0,\\
      &\partial_t(\rho E) + \partial_x\bigl(u(\rho E + p)\bigr) = 0,
    \end{aligned}
  $$

  with the closure

  $$
    p = (\gamma-1)\Bigl(\rho E - \tfrac12\rho\,u^2\Bigr),
    \quad \gamma=1.4.
  $$

2. **Available data**:
  Instead of knowing the full solution $(\rho,u,p)$ everywhere, you only “measure”:

  * The *gradient* of density via Schlieren-like data (i.e.\ $\partial_x\rho$) at many $(x,t)$ locations.
  * A time‐series of *point* pressure measurements $p(x_p,t)$ at one or a few probe locations $x_p$.

3. **Unknown targets**:
  You wish to reconstruct the full fields

  $$
    \rho(x,t),\quad u(x,t),\quad p(x,t),
  $$

  even though you’ve only directly “seen” parts of them.


#### Why inverse problems are (often) harder than forward ones

* **Ill-posedness.**  Small noise in measurements can cause large swings in the inferred solution.  Unless you regularize or “anchor” the inference with strong physics, you risk non-uniqueness or instability.
* **Data sparsity.**  Measuring full-field variables is expensive; you typically only get partial glimpses (e.g.\ Schlieren gives you gradients, not absolute values, and probes give you one point in space).
* **Nonlinearity & coupling.**  The Euler equations tightly couple $\rho$, $u$, and $p$.  Errors in one inferred field feed into others via the PDE constraints.


#### PINNs for data-driven discovery

Physics-Informed Neural Networks tackle this by training a network

$$
  (\hat\rho,\hat u,\hat p)(x,t;\theta)
$$

to simultaneously:

1. **Match measurements**

  $$
    \bigl\|\partial_x\hat\rho(x_i,t_i) - \text{measured }\partial_x\rho\bigr\|^2
    +
    \bigl\|\hat p(x_p,t_j) - \text{measured }p\bigr\|^2
    \;\approx\;0.
  $$

2. **Satisfy the PDE residuals**

  $$
    \bigl\|\partial_t \hat\rho + \partial_x(\hat\rho\,\hat u)\bigr\|^2
    +
    \bigl\|\partial_t(\hat\rho\,\hat u) + \partial_x(\hat\rho\hat u^2 + \hat p)\bigr\|^2
    + \dots
    \;\approx\;0.
  $$

3. **Honor initial/boundary conditions**

  $$
    \hat\rho(x,0)=1.0+0.2\sin(\pi x),\quad \hat u(x,0)=1.0,\quad \hat p(x,0)=1.0,
  $$

  and any boundary constraints on $x=\pm1,\ t\in[0,2]$.

During training, the network learns to reconcile the sparse, noisy data with the rigorous PDE constraints, effectively “discovering” fields that both explain the measurements and obey the laws of gas dynamics.


#### Forward vs. Inverse in our exercise

* **Forward problem** (what you just tackled): $\lambda$, PDE form, and initial/boundary data are known → solve for $u(x,t)$.
* **Inverse problem** (this exercise): PDE form and γ are known, you have only gradient and probe measurements → infer $\rho$, $u$, $p$ everywhere.

We’ll follow the strategy from Appendix B of “Physics-informed neural networks for high-speed flows”: set up a PINN that ingests the Schlieren-like $\partial_x\rho$ and point-probe $p$ data, enforces the 1D Euler residuals, and recovers the full solution fields on $[-1,1]\times[0,2]$.

In [ ]:
from utils import *

%matplotlib inline
plt.rcParams.update({"figure.figsize": (6, 4), "font.size": 12})

# Sets up GPU for PyTorch if available
device = torch.device('cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))
print(f"Using device: {device}")

In [ ]:
config = {
    'use_tqdm': True,
    'seed': 2,
}

torch.manual_seed(config['seed'])
np.random.seed(config['seed'])

In [2]:
class EulerNet(nn.Module):
    def __init__(self, layers=(2, *[120]*4, 3)):
        super().__init__()
        modules = []
        for i in range(len(layers)-1):
            modules.append(nn.Linear(layers[i], layers[i+1]))
            if i < len(layers)-2:
                modules.append(nn.Tanh())
        self.net = nn.Sequential(*modules)
    def forward(self, x):
        return self.net(x)


In [ ]:
data = load_data_euler("../data/1DEuler_data.npy")
model = EulerNet().to(device)

In [ ]:
epochs=3000
lbfgs_start=2700
lr=1e-3
weight_decay=1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2000, gamma=0.5)

losses = []
metrics = {}
model.train()

In [6]:
for key, value in data.items():
    if isinstance(value, torch.Tensor):
        data[key] = value.to(device)

## Building the physics‑informed loss

In this section we will assemble the *physics* portion of our PINN loss, enforcing that the network outputs
$(\rho,u,p)$ satisfy the 1D Euler equations in **conservative form** with the conserved energy variable $\rho E$:

$$
\begin{aligned}
&\partial_t \rho + \partial_x(\rho\,u) = 0,\\
&\partial_t(\rho\,u) + \partial_x(\rho\,u^2 + p) = 0,\\
&\partial_t(\rho E) + \partial_x\bigl(u\,(\rho E + p)\bigr) = 0,
\end{aligned}
\qquad
E = \frac{p}{(\gamma-1)} + \frac12\,\rho\,u^2.
$$

### Your tasks

1. **Energy**
   Compute the energy $E$ *before* automatic differentiation:

   $$
     E = \frac{p}{(\gamma - 1)} + \frac12\,\rho^2\,u^2.
   $$

2. **Calculate gradients**
   Use autograd to compute the following derivatives:

   $$
     \rho_t,\ \rho_x,\ u_t,\ u_x,\ p_t,\ p_x,\ E_t,\ E_x.
   $$

3. **Residuals**

   * Mass:

     $$
     F_1 = \rho_t + \partial_x(\rho\,u)
         = \rho_t + u\,\rho_x + \rho\,u_x.
     $$
   * Momentum:

     $$
     F_2
     = \partial_t(\rho\,u) + \partial_x(\rho\,u^2 + p)\\
     = \rho\,u_t + u\,\rho_t + 2\,\rho\,u\,u_x + \rho_x\,u^2 + p_x.
     $$
   * Energy (conserved):

     $$
     \begin{aligned}
     F_3
     &= \partial_t(\rho E) + \partial_x\bigl(u(\rho E + p)\bigr) \\
     &= (\rho E)_t + u_x\,(\rho E + p) + u\,\bigl[(\rho E)_x + p_x\bigr] \\
     &= (\rho E)_t + u_x\,\rho E + u_x\,p + u\,(\rho E)_x + u\,p_x \\
     &= \rho_t\,E \;+\;\rho\,E_t+u_x\,\rho E \;+\;u_x\,p \;+\;u\,\rho_x\,E \;+\;u\,\rho\,E_x \;+\;u\,p_x.
     \end{aligned}
     $$

4. **Data losses**

   * Probe pressure loss:

     $$
       \mathcal L_{\rm probe} = \frac{1}{N}\sum\bigl(p_{\rm probe}-p_{\rm pred}\bigr)^2.
     $$

5. **Final loss**

   $$
   \mathcal L
   = \underbrace{\langle F_1^2 + F_2^2 + F_3^2\rangle}_{\rm PDE}
   + \underbrace{\mathcal L_{\rm grad}}_{\rm grad.\;match}
   + \underbrace{\mathcal L_{\rm probe}}_{\rm probe}
   + \underbrace{\mathcal L_{\rm mass}}_{\rm mass\;int.}
   $$

In [ ]:
DX = 0.0999
gamma = 1.4

def physics_loss(F_pred, E_F, grads, rho_nr, rhodx_nr, grads_nr, p_probe, p_pred, rho_int0, rho_int_pred):
    """
    Computes combined loss: PDE residuals, gradient-matching, probe pressure, and mass integral constraints.
    """
    rho_F, u_F, p_F = F_pred[:, 0], F_pred[:, 1], F_pred[:, 2]
    
    # Derivatives
    rho_g, u_g, p_g, E_g = grads
    rhox_F, ux_F, px_F, Ex_F = None, None, None, None
    rhot_F, ut_F, pt_F, Et_F = None, None, None, None

    # PDE residuals
    # TODO: Calculate each term of the PDE residuals
    F1 = None
    F2 = None
    F3 = None
    loss_F = (F1.pow(2).mean() + F2.pow(2).mean() + F3.pow(2).mean())

    # gradient matching
    rho_nr_NN, rhodx_nr_NN = grads_nr
    grad_diff = ((rhodx_nr_NN - rho_nr_NN) / DX) - (rhodx_nr - rho_nr) / DX
    loss_grad = grad_diff.pow(2).mean()

    # probe pressure
    # TODO: Calculate the probe pressure loss
    loss_probe = None

    # mass integral
    loss_mass0 = (rho_int_pred - rho_int0).pow(2).mean()

    return loss_F + loss_grad + loss_probe + loss_mass0, loss_F, loss_grad, loss_probe, loss_mass0

In [ ]:
def closure():
    optimizer.zero_grad()
    F_pred = model(data['Phi_F'])

    # TODO: Calculate the total energy E_F from the other variables
    E     = None

    # Compute gradients
    # TODO: Calculate the gradients of F_pred and E with respect to data['Phi_F']
    rho_g = None
    u_g   = None
    p_g   = None
    E_g   = None
    grads = (rho_g, u_g, p_g, E_g)
    
    # For other loss terms
    rho_nr_NN = model(data['Phi_nabla'])[:, 0]
    rhodx_nr_NN = model(data['Phi_nabla_dx'])[:, 0]
    p_pred = model(data['Phi_probe'])[:, 2]
    rho_pred_m0 = model(data['Phi_mass0'])[:, 0]
    rho_int_pred = torch.trapz(rho_pred_m0, data['Phi_mass0'][:, 0])

    loss, loss_F, loss_grad, loss_probe, loss_mass0 = physics_loss(
        F_pred, E, grads,
        data['rho_nr'], data['rhodx_nr'],
        (rho_nr_NN, rhodx_nr_NN),
        data['p_probe'], p_pred,
        data['rho_int0'], rho_int_pred
    )
    loss.backward()
    metrics['loss'] = loss.item()
    metrics['loss_F'] = loss_F.item()
    metrics['loss_grad'] = loss_grad.item()
    metrics['loss_probe'] = loss_probe.item()
    metrics['loss_mass0'] = loss_mass0.item()
    return loss

In [ ]:
iterable = tqdm(range(1, epochs+1), disable=not config['use_tqdm'])
for epoch in iterable:
    if epoch == lbfgs_start:
        print("Switching to LBFGS optimizer")
        optimizer = torch.optim.LBFGS(
            model.parameters(),
            max_iter=50,
            tolerance_grad=1e-6
        )
    if epoch < lbfgs_start:
        loss = closure()
        optimizer.step()
        scheduler.step()
    else:
        loss = optimizer.step(closure)

    losses.append(
        (metrics['loss'], metrics['loss_F'], metrics['loss_grad'],
         metrics['loss_probe'], metrics['loss_mass0'])
    )
    if epoch % 200 == 0:
        print(f"Epoch {epoch:4d} | Total: {losses[-1][0]:.3e} "
              f"| PDE: {losses[-1][1]:.3e} | Gradient Matching: {losses[-1][2]:.3e}"
              f" | Probe: {losses[-1][3]:.3e} | Mass Integral: {losses[-1][4]:.3e}")


In [ ]:
loss_arr = np.array(losses)
plt.figure()
plt.semilogy(loss_arr[:,0], label='Total')
plt.semilogy(loss_arr[:,1], '--', label='PDE + Grad')
plt.semilogy(loss_arr[:,2], '--', label='Gradient Matching')
plt.semilogy(loss_arr[:,3], '--', label='Probe Pressure')
plt.semilogy(loss_arr[:,4], '--', label='Mass Integral')
plt.axvline(x=lbfgs_start, color='k', linestyle='--', label='LBFGS Start')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss (log scale)')
plt.title('Training Loss')
plt.show()

In [17]:
model.eval()
# adapt number of validation points
val_points = 500

# Create grid
x = np.linspace(-1, 1, val_points)
t = np.linspace(0, 2, val_points)
X, T = np.meshgrid(x, t)
Phi = torch.tensor(np.stack([X.flatten(), T.flatten()], axis=1), dtype=torch.float32, device=device)

with torch.no_grad():
    output = model(Phi).cpu().numpy()
rho_pred, u_pred, p_pred = output[:, 0], output[:, 1], output[:, 2]

rho_true = 1.0 + 0.2*np.sin(np.pi*(X - T))
u_true   = np.ones_like(rho_true)
p_true   = np.ones_like(rho_true)


In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(18, 8))
titles = ['rho (true)', 'u (true)', 'p (true)', 'rho (NN)', 'u (NN)', 'p (NN)']
data_list = [rho_true, u_true, p_true, rho_pred, u_pred, p_pred]
for i, data_i in enumerate(data_list):
    im = ax[i//3, i%3].imshow(data_i.reshape(val_points, val_points), aspect='auto', vmin=0, vmax=1.2)
    ax[i//3, i%3].set_title(titles[i])
    fig.colorbar(im, ax=ax[i//3, i%3])
plt.tight_layout()
plt.show()

In [ ]:
x = np.linspace(-1, 1, val_points)
t = np.linspace(0,    2, val_points)
rho_true_mesh = rho_true.reshape(val_points, val_points)
u_true_mesh   = u_true.reshape(val_points, val_points)
p_true_mesh   = p_true.reshape(val_points, val_points)
rho_pred_mesh = rho_pred.reshape(val_points, val_points)
u_pred_mesh   = u_pred.reshape(val_points, val_points)
p_pred_mesh   = p_pred.reshape(val_points, val_points)

fig, axes = plt.subplots(3, 1, figsize=(6, 9), sharex=True)
lines = []

for ax, true_m, pred_m, name in zip(
    axes,
    (rho_true_mesh, u_true_mesh, p_true_mesh),
    (rho_pred_mesh, u_pred_mesh, p_pred_mesh),
    ("rho", "u", "p")
):
    lt, = ax.plot([], [], "b-",  label=f"{name} true")
    lp, = ax.plot([], [], "r--", label=f"{name} NN")
    lines.append((lt, lp))

    ymin, ymax = true_m.min(), true_m.max()
    ymin = ymin - 0.01*(ymax-ymin)
    ymax = ymax + 0.01*(ymax-ymin)
    ax.set_xlim(x.min(), x.max())
    ax.set_ylim(ymin, ymax)

    ax.set_ylabel(name)
    ax.legend(loc="upper right")

axes[-1].set_xlabel("x")
fig.tight_layout()

def init():
    for lt, lp in lines:
        lt.set_data([], [])
        lp.set_data([], [])
    return [l for pair in lines for l in pair]

def update(frame):
    for (lt, lp), true_m, pred_m in zip(
        lines,
        (rho_true_mesh, u_true_mesh, p_true_mesh),
        (rho_pred_mesh, u_pred_mesh, p_pred_mesh)
    ):
        lt.set_data(x, true_m[frame, :])
        lp.set_data(x, pred_m[frame, :])
    axes[0].set_title(f"t = {t[frame]:.3f}")
    return [l for pair in lines for l in pair]

ani = FuncAnimation(
    fig, update,
    frames=val_points,
    init_func=init,
    blit=True,
    interval=50
)

HTML(ani.to_jshtml())